In [2]:
import pandas as pd
import numpy as np
import json
#from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import requests

In [2]:
1 - 1.06751/2.06751

0.48367359770932183

In [3]:
scale_factor = 1e6

In [1]:
import os

In [6]:
import imageio
import os

images = []
for f in os.listdir("img/"):
    #print(filename)
    filename = "img/" + f
    images.append(imageio.imread(filename))
imageio.mimsave('religion_diversity.gif', images, duration=1)

#### Total population

In [30]:
population_total = 1324171354
population_scaled = round(population_total/scale_factor)
population_scaled

1324

#### Split by gender

In [24]:
male_population_total = 1.06751/2.06751 * population_total
female_population_total = population_total - male_population_total
male_population_scaled = round(1.06751/2.06751 * population_scaled)
female_population_scaled = population_scaled - male_population_scaled

#### Split by religion

In [61]:
hindus_percentage = 79.80
muslims_percentage = 14.23
christians_percentage = 2.30
sikhs_percentage = 1.72
buddhists_percentage = 0.70
jains_percentage = 0.37
others_percentage = 0.66
notstated_percentage = 0.24

hindus_population_total = round(population_total * hindus_percentage/100)
muslims_population_total = round(population_total * muslims_percentage/100)
christians_population_total = round(population_total * christians_percentage/100)
sikhs_population_total = round(population_total * sikhs_percentage/100)
buddhists_population_total = round(population_total * buddhists_percentage/100)
jains_population_total = round(population_total * jains_percentage/100)
others_population_total = round(population_total * others_percentage/100)
notstated_population_total = round(population_total * notstated_percentage/100)

hindus_population_scaled = round(population_scaled * hindus_percentage / 100)
muslims_population_scaled = round(population_scaled * muslims_percentage / 100)
christians_population_scaled = round(population_scaled * christians_percentage / 100)
sikhs_population_scaled = round(population_scaled * sikhs_percentage / 100)
buddhists_population_scaled = round(population_scaled * buddhists_percentage / 100)
jains_population_scaled = round(population_scaled * jains_percentage / 100)
others_population_scaled = round(population_scaled * others_percentage / 100)
notstated_population_scaled = round(population_scaled * notstated_percentage / 100)

religion_thresholds = []
acc = 0

for i in range(8):
    if i==1:
        acc += hindus_population_scaled
    elif i==2:
        acc += muslims_population_scaled
    elif i==3:
        acc += christians_population_scaled
    elif i==4:
        acc += sikhs_population_scaled
    elif i==5:
        acc += buddhists_population_scaled
    elif i==6:
        acc += jains_population_scaled
    elif i==7:
        acc += others_population_scaled
    elif i==8:
        acc += notstated_population_scaled
    
    religion_thresholds.append(acc)
religion_thresholds

#### Split by states

In [288]:
dfsp = pd.read_csv("../bubble_chart_v4/data/state_population.csv", sep="\t", usecols=[1,3])
dfsp.columns = [["state", "percentage"]]
dfsp["population_total"] = population_total
dfsp["population_scaled"] = population_scaled
dfsp["state_population_total"] = round(dfsp["percentage"] * dfsp["population_total"] / 100)
dfsp["state_population_scaled"] = round(dfsp["percentage"] * dfsp["population_scaled"] / 100)
dfsp["state_population_scaled"].replace(0,1, inplace=True)

#Decrement 1 for top 3 populous states to accomodate bottom 3 populous states
dfsp.loc[dfsp["state"].isin(["Uttar Pradesh", "Maharashtra", "Bihar"]), "state_population_scaled"] = dfsp.loc[dfsp["state"].isin(["Uttar Pradesh", "Maharashtra", "Bihar"]), "state_population_scaled"] - 1

state_population = []
for state in dfsp["state"]:
    state_population.append([state for pop in range(int(dfsp[dfsp["state"]==state]["state_population_scaled"].values[0]))])

state_population = [item for sublist in state_population for item in sublist]

#### State literacy

In [308]:
dfsl = pd.read_csv("../bubble_chart_v4/data/state_literacy.csv", sep="\t", usecols=[1,2])
dfsl.columns = [["state", "literacy"]]
dfspl = dfsp.set_index("state").join(dfsl.set_index("state")).reset_index()
dfspl["literate_population_scaled"] = round(dfspl["state_population_scaled"] * dfspl["literacy"] / 100)
dfspl["illiterate_population_scaled"] = dfspl["state_population_scaled"] - dfspl["literate_population_scaled"]

literate_population = []
for state in dfspl["state"]:
    state_illiterate_population_scaled = int(dfspl[dfspl["state"]==state]["illiterate_population_scaled"].values[0])
    literate_population.append([0] * state_illiterate_population_scaled)
    state_literate_population_scaled = int(dfspl[dfspl["state"]==state]["literate_population_scaled"].values[0])
    literate_population.append([1] * state_literate_population_scaled)

literate_population = [item for sublist in literate_population for item in sublist]

#### Age group

In [178]:
"""
https://www.indexmundi.com/india/demographics_profile.html
0-14 years: 27.34% (male 186,087,665/female 164,398,204)
15-24 years: 17.9% (male 121,879,786/female 107,583,437)
25-54 years: 41.08% (male 271,744,709/female 254,834,569)
55-64 years: 7.45% (male 47,846,122/female 47,632,532)
65 years and over: 6.24% (male 37,837,801/female 42,091,086) (2017 est.)
"""
agegroup_percentage = [27.34, 17.9, 41.08, 7.45, 6.24]
agegroup_population_scaled = [round(x*population_scaled/100) for x in agegroup_percentage]
agegroup_population = []
for i in range(len(agegroup_population_scaled)):
    agegroup_population.append([i] * agegroup_population_scaled[i])

agegroup_population = [item for sublist in agegroup_population for item in sublist]

### Rural/urban divide

In [188]:
#33.5% of total population
#https://www.indexmundi.com/india/demographics_profile.html
rural_population_percentage = 33.5
rural_population_scaled = round(population_scaled * rural_population_percentage / 100)
urban_population_scaled = population_scaled - rural_population_scaled

ruralurban_population = []
ruralurban_population.append([0] * rural_population_scaled) #Rural
ruralurban_population.append([1] * urban_population_scaled) #Urban

ruralurban_population = [item for sublist in ruralurban_population for item in sublist]

### Languages

In [233]:
#https://en.wikipedia.org/wiki/List_of_languages_by_number_of_native_speakers_in_India
dfl = pd.read_csv("../bubble_chart_v4/data/language2.csv", sep="\t", header=None, usecols=[1,8])
dfl.columns = [["language", "percentage"]]
dfl["population_scaled"] = population_scaled
dfl["language_population_scaled"] = round(dfl["population_scaled"] * dfl["percentage"] / 100)
dfl.loc[dfl["language"]=="Hindi", "language_population_scaled"] = dfl[dfl["language"]=="Hindi"]["language_population_scaled"] - 3
dfl
language_population = []
for i in range(len(dfl)):
    language_population.append([dfl.iloc[i]["language"] for j in range(int(dfl.iloc[i]["language_population_scaled"]))])
language_population = [item for sublist in language_population for item in sublist]

#### Unemployment rate

In [315]:
"""
#3.52% of total population [2017]
#https://tradingeconomics.com/india/unemployment-rate
#https://www.statista.com/statistics/271330/unemployment-rate-in-india/
unemployed_population_percentage = 3.52
unemployed_population_scaled = round(population_scaled * unemployed_population_percentage / 100)
employed_population_scaled = population_scaled - unemployed_population_scaled

unemployed_population = []
unemployed_population.append([0] * unemployed_population_scaled) #Unemployed
unemployed_population.append([1] * employed_population_scaled) #Employed

unemployed_population = [item for sublist in unemployed_population for item in sublist]
"""
print()

#### Sanitation

In [238]:
#https://data.worldbank.org/indicator/SH.STA.ODFC.ZS?locations=IN
unsanitation_population_percentage = 39.8 #Open defecation
unsanitation_population_scaled = round(population_scaled * unsanitation_population_percentage / 100)
sanitation_population_scaled = population_scaled - unsanitation_population_scaled

sanitation_population = []
sanitation_population.append([0] * unsanitation_population_scaled) #Open
sanitation_population.append([1] * sanitation_population_scaled) 

sanitation_population = [item for sublist in sanitation_population for item in sublist]

#### Poverty

In [310]:
#https://en.wikipedia.org/wiki/Poverty_in_India - As of 2014, 58% of the total population were living on less than $3.10 per day.
poverty_population_percentage = 58
#poverty_population_scaled
poverty_population_scaled = round(population_scaled * poverty_population_percentage / 100)
nonpoverty_population_scaled = population_scaled - poverty_population_scaled

poverty_population = []
poverty_population.append([0] * poverty_population_scaled) #Poverty
poverty_population.append([1] * nonpoverty_population_scaled) 

poverty_population = [item for sublist in poverty_population for item in sublist]

#### Employment sectors

In [318]:
#### Employment sectors
#https://www.statista.com/statistics/271320/distribution-of-the-workforce-across-economic-sectors-in-india/
#agriculture: 42.74, industry:23.79, services: 33.48, Unemployed:3.52%
unemployed_percentage = 3.52
agriculture_percentage = (100-unemployed_percentage)*42.74/100
industry_percentage = (100-unemployed_percentage)*23.79/100
services_percentage = (100-unemployed_percentage)*33.48/100

agriculture_population_scaled = round(population_scaled * agriculture_percentage / 100) 
industry_population_scaled = round(population_scaled * industry_percentage / 100) 
services_population_scaled = round(population_scaled * services_percentage / 100) 
employed_population_scaled = agriculture_population_scaled + industry_population_scaled + services_population_scaled
unemployed_population_scaled = population_scaled - employed_population_scaled

sector_population = []
sector_population.append([0] * agriculture_population_scaled)
sector_population.append([1] * industry_population_scaled)
sector_population.append([2] * services_population_scaled)
sector_population.append([3] * unemployed_population_scaled)

sector_population = [item for sublist in sector_population for item in sublist]

#### Tax payers

In [311]:
#percent_tax_payers = 2

tax_population_percentage = 2
tax_population_scaled = round(population_scaled * tax_population_percentage / 100)
nontax_population_scaled = population_scaled - tax_population_scaled

tax_population = []
tax_population.append([0] * tax_population_scaled) #Tax payers
tax_population.append([1] * nontax_population_scaled) 

tax_population = [item for sublist in tax_population for item in sublist]

In [323]:
df = pd.DataFrame()
idx = 0

for i in range(population_scaled):
    
    #####################################
    ############# Gender ################
    #####################################
    if i<= male_population_scaled:
        # Male
        df.at[idx, "gender"] = "male"
    else:
        # Female
        df.at[idx, "gender"] = "female"
    #####################################

        
    ########################################################
    ##################### Religion #########################
    ########################################################
    if i>=religion_thresholds[0] and i<=religion_thresholds[1]:
        df.at[idx, "religion"] = "Hindu"
    elif i>religion_thresholds[1] and i<=religion_thresholds[2]:
        df.at[idx, "religion"] = "Muslim"    
    elif i>religion_thresholds[2] and i<=religion_thresholds[3]:
        df.at[idx, "religion"] = "Christian"            
    elif i>religion_thresholds[3] and i<=religion_thresholds[4]:
        df.at[idx, "religion"] = "Sikh"            
    elif i>religion_thresholds[4] and i<=religion_thresholds[5]:
        df.at[idx, "religion"] = "Buddhist"            
    elif i>religion_thresholds[5] and i<=religion_thresholds[6]:
        df.at[idx, "religion"] = "Jain"            
    elif i>religion_thresholds[6] and i<=religion_thresholds[7]:
        df.at[idx, "religion"] = "Other"            
    elif i>religion_thresholds[7]:
        df.at[idx, "religion"] = "Not stated"            
    ##############################################################
    ##############################################################
        
    idx+=1
df["state"] = state_population
df["agegroup"] = agegroup_population[:len(df)]
df["ruralurban"] = ruralurban_population
df["language"] = language_population
df["literacy"] = literate_population
df["employment"] = sector_population
df["poverty"] = poverty_population
df["tax"] = tax_population
df["sanitation"] = sanitation_population
df.reset_index(inplace=True)
df.rename(columns={"index":"id"}, inplace=True)
df.to_csv("../bubble_chart_v4/data/india_demographics.csv", index=False)

### Random ordering

In [14]:
df = pd.read_csv("../code/data/india_demographics.csv")

In [15]:
df.columns

Index(['id', 'gender', 'religion', 'state', 'agegroup', 'ruralurban',
       'language', 'literacy', 'employment', 'poverty', 'tax', 'sanitation'],
      dtype='object')

In [17]:
dfr = pd.DataFrame() #df.copy()
dfr["gender"] = df["gender"].sample(frac=1)
dfr["religion"] = df["religion"].sample(frac=1)
dfr["agegroup"] = df["agegroup"].sample(frac=1)
dfr["ruralurban"] = df["ruralurban"].sample(frac=1)
dfr["language"] = df["language"].sample(frac=1)
dfr["employment"] = df["employment"].sample(frac=1)
dfr["poverty"] = df["poverty"].sample(frac=1)
dfr["tax"] = df["tax"].sample(frac=1)
dfr["sanitation"] = df["sanitation"].sample(frac=1)
dfr[["state","literacy"]] = df[["state","literacy"]].sample(frac=1)

In [19]:
dfr.to_csv("../code/data/india_demographics_shuffled.csv", index=False)

In [332]:
df["language"].unique()

array(['Hindi', 'Bengali', 'Marathi', 'Telugu', 'Tamil', 'Gujarati',
       'Urdu', 'Kannada', 'Odia', 'Malayalam', 'Punjabi', 'Assamese',
       'Maithili', 'Bhili/Bhilodi', 'Santali', 'Kashmiri', 'Gondi',
       'Nepali', 'Sindhi', 'Dogri', 'Konkani', 'Kurukh', 'Khandeshi',
       'Tulu', 'Meitei (Manipuri)', 'Bodo', 'Khasi', 'Ho', 'Mundari',
       'Garo', 'Tripuri'], dtype=object)

In [330]:
df["state"].unique()

array(['Uttar Pradesh', 'Maharashtra', 'Bihar', 'West Bengal',
       'Madhya Pradesh', 'Tamil Nadu', 'Rajasthan', 'Karnataka', 'Gujarat',
       'Andhra Pradesh', 'Odisha', 'Telangana', 'Kerala', 'Jharkhand',
       'Assam', 'Punjab', 'Chhattisgarh', 'Haryana', 'Delhi',
       'Jammu and Kashmir', 'Uttarakhand', 'Himachal Pradesh', 'Tripura',
       'Meghalaya', 'Manipur', 'Nagaland', 'Goa', 'Arunachal Pradesh',
       'Puducherry', 'Mizoram', 'Chandigarh', 'Sikkim',
       'Andaman and Nicobar Islands', 'Dadra and Nagar Haveli',
       'Daman and Diu', 'Lakshadweep'], dtype=object)

In [4]:
df = pd.read_csv("../code/data/india_demographics_shuffled.csv")


array(['Gujarati', 'Bengali', 'Marathi', 'Bhili/Bhilodi', 'Hindi',
       'Telugu', 'Kannada', 'Punjabi', 'Konkani', 'Urdu', 'Maithili',
       'Tamil', 'Odia', 'Assamese', 'Malayalam', 'Gondi', 'Santali',
       'Bodo', 'Sindhi', 'Dogri', 'Khasi', 'Nepali', 'Kashmiri',
       'Tripuri', 'Meitei (Manipuri)', 'Garo', 'Ho', 'Khandeshi', 'Tulu',
       'Kurukh', 'Mundari'], dtype=object)

In [5]:
len(df["language"].unique())

31